## 교차검증

In [18]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [16]:
iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=20190328)

# 폴드 세트를 분리하는 KFold 객체
# 폴드 세트 별로 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 크기: ', features.shape[0])

붓꽃 데이터 크기:  150


## 1. KFold
- Decision Tree Classifier
- 붓꽃 데이터

In [17]:
n_iter = 0
# KFold의 .split()은 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
for train_index, test_index in kfold.split(features):
    
    # train_index, test_index 를 이용 -> 학습용, 검증용 테스트 데이터 slicing
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    # Accuracy 계산
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    cv_accuracy.append(accuracy)
    train_size = X_train.shape[0]
    test_size = y_train.shape[0]
    print('\n#{} 교차 검증 정확도: {}, 학습 데이터 크기: {}, 검증 데이터 크기: {}'\
          .format(n_iter, accuracy, train_size, test_size))
    print('#{} 검증 세트 인덱스: \n{}'.format(n_iter, test_index))
    
# 개별 iteration 별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차 검증 정확도: 1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 120
#1 검증 세트 인덱스: 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도: 1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 120
#2 검증 세트 인덱스: 
[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도: 0.9, 학습 데이터 크기: 120, 검증 데이터 크기: 120
#3 검증 세트 인덱스: 
[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도: 0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 120
#4 검증 세트 인덱스: 
[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도: 0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 120
#5 검증 세트 인덱스: 
[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.91332


## 2. Stratified KFold

In [19]:
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

### 2-1. Kfold로 나눈 학습/검증 레이블 데이터 분포도

In [23]:
kfold = KFold(n_splits=3)
n_iter = 0
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차검증:'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차검증:
학습 레이블 데이터 분포:
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차검증:
학습 레이블 데이터 분포:
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차검증:
학습 레이블 데이터 분포:
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


### 2-2. stratified kfold
- stratified는 imbalanced data 의 kfold 를 위한 것이므로, .split() 메소드에 irid_df 와 iris_df['label'] 을 함께 넣어주어야한다.
- 다음 결과를 보면 분포도가 동일하게 할당 됐음을 알 수 있다.

In [25]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차검증:'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차검증:
학습 레이블 데이터 분포:
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    17
Name: label, dtype: int64
## 교차검증:
학습 레이블 데이터 분포:
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    17
Name: label, dtype: int64
## 교차검증:
학습 레이블 데이터 분포:
 2    34
1    34
0    34
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    16
1    16
0    16
Name: label, dtype: int64


### 2-3. Stratified KFold 를 이용한 Decision Tree Classifier


In [30]:
dt_clf = DecisionTreeClassifier(random_state=20190328)

skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy = []

for train_index, test_index in skfold.split(features, label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    cv_accuracy.append(accuracy)
    train_size = X_train.shape[0]
    test_size = y_train.shape[0]
    print('\n#{} 교차 검증 정확도: {}, 학습 데이터 크기: {}, 검증 데이터 크기: {}'\
          .format(n_iter, accuracy, train_size, test_size))
    print('#{} 검증 세트 인덱스: \n{}'.format(n_iter, test_index))
    
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차 검증 정확도: 0.9804, 학습 데이터 크기: 99, 검증 데이터 크기: 99
#1 검증 세트 인덱스: 
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116]

#2 교차 검증 정확도: 0.9216, 학습 데이터 크기: 99, 검증 데이터 크기: 99
#2 검증 세트 인덱스: 
[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133]

#3 교차 검증 정확도: 0.9792, 학습 데이터 크기: 102, 검증 데이터 크기: 102
#3 검증 세트 인덱스: 
[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9604


## 3. 교차 검증을 편하게 해주는 API
### 3-1. cross_val_score
- cross_val_score 의 경우 Classification 이면, Stratified KFold, Regression 이면, KFold

In [34]:
from sklearn.model_selection import cross_val_score

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=20190328)

data = iris_data.data
label = iris_data.target

scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.9667 0.9667 0.9    0.9667 1.    ]
평균 검증 정확도: 0.96


### 3-2. cross_validate
- corss_val_score 의 경우 한가지 지표를 가지고 교차검증을 수행하지만, cross_validate 의 경우 여러 평가지표를 수행할 수 있다.

In [47]:
from sklearn.model_selection import cross_validate

validations = cross_validate(dt_clf, data, label, scoring=['accuracy', 'balanced_accuracy'] ,cv=5)
validations

{'fit_time': array([0.00088024, 0.00036192, 0.00039887, 0.00036097, 0.00038695]),
 'score_time': array([0.0019238 , 0.00063896, 0.00067306, 0.00075889, 0.00064588]),
 'test_accuracy': array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ]),
 'train_accuracy': array([1., 1., 1., 1., 1.]),
 'test_balanced_accuracy': array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ]),
 'train_balanced_accuracy': array([1., 1., 1., 1., 1.])}

### 3-3. GridSearchCV
- 교차 검증과 하이퍼 파라미터 튜닝을 한번에

In [51]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.2, random_state=0)

dt_clf = DecisionTreeClassifier()

grid_parameters = {
    'max_depth': [1, 2, 3],
    'min_samples_split': [2, 3]
}

grid_dtree = GridSearchCV(dt_clf, param_grid=grid_parameters, cv=5, refit=True)

grid_dtree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df

/Users/emjayahn/.pyenv/versions/3.7.0/envs/dss/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000303,0.000069,0.000287,0.000181,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.68,0.68,0.708333,...,0.691667,0.013624,5,0.694737,0.694737,0.687500,0.687500,0.693878,0.691670,0.003419
1,0.000255,0.000012,0.000192,0.000007,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.68,0.68,0.708333,...,0.691667,0.013624,5,0.694737,0.694737,0.687500,0.687500,0.693878,0.691670,0.003419
2,0.000263,0.000008,0.000190,0.000007,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.92,0.92,1.000000,...,0.941667,0.033560,1,0.957895,0.968421,0.947917,0.958333,0.969388,0.960391,0.007893
3,0.000271,0.000015,0.000201,0.000019,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.92,0.92,1.000000,...,0.941667,0.033560,1,0.957895,0.968421,0.947917,0.958333,0.969388,0.960391,0.007893
4,0.000311,0.000037,0.000227,0.000047,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.92,0.92,1.000000,...,0.925000,0.041010,4,0.978947,0.978947,0.958333,0.979167,0.989796,0.977038,0.010242
5,0.000273,0.000015,0.000190,0.000008,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.96,0.92,1.000000,...,0.933333,0.043155,3,0.978947,0.978947,0.958333,0.979167,0.989796,0.977038,0.010242
